In [8]:
import qutip
import numpy as np
from scipy.linalg import expm

First,  by inspection construct a target unitary for cyclic_3 swap.

$U|a, b, c\rangle = |b,c,a\rangle$.

\begin{equation}
\begin{bmatrix}
1& 0& 0& 0& 0& 0& 0& 0\\
0& 0& 0& 0& 1& 0& 0& 0\\
0& 1& 0& 0& 0& 0& 0& 0\\
0& 0& 0& 0& 0& 1& 0& 0\\
0& 0& 1& 0& 0& 0& 0& 0\\
0& 0& 0& 0& 0& 0& 1& 0\\
0& 0& 0& 1& 0& 0& 0& 0\\
0& 0& 0& 0& 0& 0& 0& 1
\end{bmatrix}

\begin{bmatrix}
a\\
b\\
c\\
d\\
e\\
f\\
g\\
h
\end{bmatrix}
=
\begin{bmatrix}
a\\
e\\
b\\
f\\
c\\
g\\
d\\
h
\end{bmatrix}
\end{equation}

In [190]:
def build_circulator_U(phi_ab=0, phi_ac=0, phi_bc=np.pi/2, g_ab=1.0, g_ac=1.0, g_bc=1.0):
    #TODO: when sweeping phase parameters leave time-independent
    # when sweeping coupling parameters make time-dependet i.e gaussian shaped pusle
    #build raising and lowering operations
    a = qutip.operators.create(N=2)
    I2 = qutip.operators.identity(2)
    A = qutip.tensor(a, I2, I2)
    B = qutip.tensor(I2, a, I2)
    C = qutip.tensor(I2, I2, a)

    #construct circulator Hamiltonian
    H_ab = np.exp(1j * phi_ab)*A*B.dag() + np.exp(-1j * phi_ab)*A.dag()*B
    H_ac = np.exp(1j * phi_ac)*A*C.dag() + np.exp(-1j * phi_ac)*A.dag()*C
    H_bc = np.exp(1j * phi_bc)*B*C.dag() + np.exp(-1j * phi_bc)*B.dag()*C
    H = g_ab*H_ab + g_ac*H_ac + g_bc*H_bc
    
    #time evolution, if time dependent need to use master-equation
    #qutip.mesolve()
    # U = expm(1j*np.array(H))
    U = (1j*H).expm()
    return U

# #construct an example initial state
# inita = qutip.basis(2, 1)
# initb = qutip.basis(2, 0)
# initc = qutip.basis(2, 0)
# init = qutip.tensor(inita, initb, initc)
# #print(init)

# U = build_circulator_U()
# #apply unitary to initial state
# new_state = U*init
# print(new_state)


# #expected
# expected = qutip.tensor(initb, initc, inita)
# print(expected.overlap(new_state))

In [181]:
expected_U = qutip.Qobj(dims = [[2, 2, 2], [2, 2, 2]],inpt=(1.0+0j)* np.array([[1,0,0,0,0,0,0,0],[0,0,0,0,1,0,0,0],[0,1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0],[0,0,1,0,0,0,0,0],[0,0,0,0,0,0,1,0],[0,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,1]]))
# gk = [1,1,1]
# template = build_circulator_U(g_ab=gk[0], g_ac=gk[1], g_bc=gk[2])
# #1- np.abs(np.sum(np.matmul(template.full(), expected.conj().full())))/ (2*np.log2(expected.shape[0]))
# np.real(((expected_U.full() - template.full()) * (expected_U.full() - template.full()).conj()).mean() ** 0.5)
# print(expected_U)

In [192]:
from scipy import optimize as opt
from itertools import product
#initial guess
gk = [1,1,1]

#Option 1. Optimize inner product of quantum states
#Problem is that we want the expected inner product to be the same over all states not just 1 at a time
#construct an example initial state
def build_init_state(a,b,c):
    inita = qutip.basis(2, a)
    initb = qutip.basis(2, b)
    initc = qutip.basis(2, c)
    init = qutip.tensor(inita, initb, initc)
    return init

#expected
#expected_state = qutip.tensor(initb, initc, inita)
init = build_init_state(0,1,1)
expected_state = expected_U*init
# print(expected_state)


#can we improve this by iterating over all basis states?
def foo(gk):
    sum = 0
    k = 0
    for a,b,c in product([0,1], repeat=3):
        init = build_init_state(a,b,c)
        inner_product = expected_state.overlap(build_circulator_U(g_ab=gk[0], g_ac=gk[1], g_bc=gk[2])*init)
        sum += 1 - np.real(np.conj(inner_product) * inner_product)
        k+=1
    return sum/k

#Option 2. Optmize target unitary difference
#The problem with this method is that it is not clear how to define difference cost function between 2 matrices, and they might be similar operations but don't look similar because differ by 1Q gates or with complex phase
# expected = qutip.Qobj(np.array([[1,0,0,0,0,0,0,0],[0,0,0,0,1,0,0,0],[0,1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0],[0,0,1,0,0,0,0,0],[0,0,0,0,0,0,1,0],[0,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,1]]))
# def foo2(gk):
#     template = build_circulator_U(g_ab=gk[0], g_ac=gk[1], g_bc=gk[2])
#     return np.real(((expected.full() - template.full()) * (expected.full() - template.full()).conj()).mean() ** 0.5)
#     #return 1- np.abs(np.sum(np.matmul(template.full(), expected.conj().full())))/ (2*np.log2(expected.shape[0]))

opt_result = opt.minimize(fun=foo, x0=gk)
print(opt_result.fun)
print(opt_result.x)

# gk = opt_result.x
# U = build_circulator_U(g_ab=gk[0], g_ac=gk[1], g_bc=gk[2])
# from pprint import pprint
# pprint(np.matrix(U))

0.875
[1. 1. 1.]


This feels like evidence that for time-independent parameters, you can't build the target unitary. You can get near-exact for states at a time, but if we iterate over just the most basic basis states we can't get a unitary that works for them all simultaneously.